# Autoencoding Dates in CS:GO Data
Projecting large dimensional time vectors into smaller vectors to be graphed

In [1]:
% matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.layers import Input, Dense
from keras.models import Model

In [88]:
# open in data file
dat = pd.read_csv('Time_Matrix.csv')

## construct input matrix
inps = np.transpose(np.array(dat.ix[:,1:(dat.shape[1]-1)] ) )/5 # transpose to make it for each date we have a vector of teams playing
# need to also add the vector of idx number
dates = np.reshape(np.arange(dat.shape[1]-2),(dat.shape[1]-2,1))
dates = dates/np.max(dates)

inps = np.concatenate( (inps, dates), axis = 1)

print(inps.shape)

(558, 1363)


### Use autoencoder now


In [90]:
''' define the model architecture '''
# this returns a tensor
inputs = Input(shape=(np.shape(inps)[1],))

encoding_dim = 3
# a layer instance is callable on a tensor, and returns a tensor
ENCOD = Dense(encoding_dim, activation='relu')(inputs)
DECOD = Dense(np.shape(inps)[1], activation='relu')(ENCOD)
#predictions = Dense(10, activation='softmax')(x)

# this creates a model that includes
# the Input layer and three Dense layers
autoencoder = Model(input=inputs, output=DECOD)

# this model maps an input to its encoded representation
encoder = Model(inputs, output = ENCOD)

# create a placeholder for an encoded (N-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
#decoder_layer = autoencoder.layers[-1]
# create the decoder model
#decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

### compile the model...
autoencoder.compile(optimizer='rmsprop', loss='mse')

Train this model now!!!
- Many epochs have been run but showing only 1 for sake of presentation

In [125]:
autoencoder.fit(inps, inps,
                nb_epoch=1,
                batch_size=1,
                shuffle=True )# ,validation_split=0.3) # obviously increase the epoch's if not trained - so far about 200 has been ran

Epoch 1/1
558/558 [==============================] - 0s - loss: 0.0048     


In [93]:
''' this is for testing how well at reconstructing inputs the net is '''
#autoencoder.predict(rosterz) - # uncomment if needed

' this is for testing how well at reconstructing inputs the net is '

In [94]:
''' save the predictions... '''
encoded_inputs = pd.DataFrame(encoder.predict(inps))

In [112]:
''' make names for months '''
encoded_inputs['Month'] =  np.mod(np.reshape(np.arange(dat.shape[1]-2),(dat.shape[1]-2,1)), 12) 
mth = []
for i in range(len(encoded_inputs)):
    if(encoded_inputs['Month'][i] == 0):
        mth.append('January')
    elif(encoded_inputs['Month'][i] == 1):
        mth.append('February')
    elif(encoded_inputs['Month'][i] == 2):
        mth.append('March')
    elif(encoded_inputs['Month'][i] == 3):
        mth.append('April')
    elif(encoded_inputs['Month'][i] == 4):
        mth.append('May')
    elif(encoded_inputs['Month'][i] == 5):
        mth.append('June')
    elif(encoded_inputs['Month'][i] == 6):
        mth.append('July')
    elif(encoded_inputs['Month'][i] == 7):
        mth.append('August')
    elif(encoded_inputs['Month'][i] == 8):
        mth.append('September')
    elif(encoded_inputs['Month'][i] == 9):
        mth.append('October')
    elif(encoded_inputs['Month'][i] == 10):
        mth.append('November')
    elif(encoded_inputs['Month'][i] == 11):
        mth.append('December')

In [119]:
# save time matrix finalized:
encoded_inputs['IDX'] = np.reshape(np.arange(dat.shape[1]-2),(dat.shape[1]-2,1))
encoded_inputs['IDX']=encoded_inputs['IDX'].apply(str)

encoded_inputs['MTH']  = mth
encoded_inputs['COMB'] = encoded_inputs['MTH'] + encoded_inputs['IDX'] 
encoded_inputs.to_csv('ENCODED TIME.csv')

### Use plotly to show some of the autoencoded dates...

In [123]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly 
plotly.tools.set_credentials_file(username='', api_key='') # removed plotly membership for privacy now...

trace1 = go.Scatter3d(
    x=encoded_inputs.ix[:100,0],
    y=encoded_inputs.ix[:100,1],
    z=encoded_inputs.ix[:100,2],
    mode='markers',
    text =  encoded_inputs.ix[:100,6]  , 
    marker=dict(
        size=4,
        line=dict(
            color='rgba(217, 217, 217, 1)',
            width=0.5
        ),
        opacity=1
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

The key takeaway is that the data was randomly generated, so it is not likely to capture any visible relationships. Once we use the real data this should result in a meaningful mapping

### Once trained we can save the autoencoder for future use

In [124]:
# save the trained model file:

from keras.models import load_model

autoencoder.save('time_autoencoder.h5')  # creates a HDF5 file 'my_model.h5'

#del model  # deletes the existing model - uncomment when needed

# loads in a compiled model
# identical to the previous one - uncomment when needed
#autoencoder = load_model('roster_autoencoder.h5')